In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [ ]:
# Creamos los DataFrames y les asignamos un nombre

bank_data = pd.read_csv('./Datos/bank-additional.csv')
customers_data = pd.read_excel('./Datos/customer-details.xlsx')


In [ ]:
# Primero vamos a tratar el DataFrame "bank_data"

# Reemplaza comas por puntos y convierte a float
cols_to_fix = ['cons.price.idx', 'cons.conf.idx', 'euribor3m']
for col in cols_to_fix:
    bank_data[col] = bank_data[col].str.replace(',', '.').astype(float)

In [ ]:
# Hay que arreglar el formato de la fecha

meses_es = {
    'enero': 'January', 'febrero': 'February', 'marzo': 'March',
    'abril': 'April', 'mayo': 'May', 'junio': 'June',
    'julio': 'July', 'agosto': 'August', 'septiembre': 'September',
    'octubre': 'October', 'noviembre': 'November', 'diciembre': 'December'
}

def traducir_fecha(fecha):
    if pd.isna(fecha):
        return pd.NaT
    for esp, eng in meses_es.items():
        fecha = fecha.replace(esp, eng)
    try:
        return pd.to_datetime(fecha, format='%d-%B-%Y')
    except ValueError:
        return pd.NaT  # Por si falla el formato

bank_data['date'] = bank_data['date'].apply(traducir_fecha)


In [ ]:
# La visualiazación de los DataFrames la he hecho con los siguientes métodos

bank_data.head()
customers_data.head()

bank_data.info()
customers_data.info()

# No lo dejo en el fichero para una mayor limpieza

In [ ]:
customers_data.head()

In [ ]:
# Viendo que el identificador de cliente de la tabla de "customers_data" está presente como identificador de operaciones en "bank_data"
# podemos plantear la union de las tablas

# La columna que no aporta información ("Unnamed:0") la eliminaremos una vez hecha la unión, para no hacerlo dos veces

# Usaremos un left join siendo la izda bank_data, ya que queremos mantener todos sus registros y es customers_data la que enriquece a bank_data

tabla_final = bank_data.merge(customers_data, left_on='id_', right_on='ID', how='left')
tabla_final.head()

In [ ]:
# Ahora vamos a ver cómo está la tabla final

# Revisión general
print("Tamaño de tabla_final:", tabla_final.shape)

# Verifica columnas clave
print("\n¿'id_' es única en bank_data? ", bank_data['id_'].is_unique)
print("¿'ID' es única en customers_data? ", customers_data['ID'].is_unique)

# Nulos por columna (solo las más relevantes)
print("\nNulos por columna:")
print(tabla_final[['Income', 'Kidhome', 'Teenhome', 'NumWebVisitsMonth']].isna().sum())

# ¿Hay clientes sin datos de customers_data?
missing_customers = tabla_final['Income'].isna().sum()
print(f"\nClientes en tabla_final sin datos demográficos: {missing_customers}")


In [ ]:
tabla_final.sample(10)

In [ ]:
# Eliminamos la columna innecesaria

tabla_final = tabla_final.loc[:, ~tabla_final.columns.str.startswith('Unnamed')]
tabla_final.head(1)

In [ ]:
# Quizá no sea útil del todo, pero para estructurar mentalmente el análisis, creo que puede ser útil hacer que el id sea el índice del DataFrame

tabla_final.set_index(["id_"], inplace = True)

In [ ]:
tabla_final.index.name = 'id'

In [ ]:
pd.set_option('display.max_columns', None)
tabla_final.head(3)

In [ ]:
tabla_final.shape

In [ ]:
# Atendiendo a realizar un análisis útil y con sentido, tenemos que tratar el tema de los valores nulos. Pienso que eliminar toda fila en la que haya un valor nulo es demasiado radical, 
# y por eso creo más sensato estudiar los datos de forma visual y empírica para tomar una decisión

In [ ]:
tabla_final['nulos_totales'] = tabla_final.isna().sum(axis=1)

import matplotlib.pyplot as plt

tabla_final['nulos_totales'].hist(bins=30)
plt.title("Distribución de nulos por fila")
plt.xlabel("Cantidad de valores nulos")
plt.ylabel("Número de filas")
plt.show()


In [ ]:
tabla_final['nulos_totales'].describe()


In [ ]:
# Atendiendo a la distribución y frecuencia de nulos: 

# Se eliminan las filas con más de 6 valores nulos, al representar registros con baja calidad informativa. Esta decisión se basa en la distribución de nulos observada, 
# que muestra una acumulación significativa de casos con 6 o más ausencias. El objetivo es mantener una base sólida para el análisis descriptivo y visual posterior sin 
# comprometer la integridad analítica del proyecto.

In [ ]:
# Aplicar el filtro
tabla_final_limpia = tabla_final[tabla_final['nulos_totales'] <= 6].copy()

# Eliminar la columna auxiliar
tabla_final_limpia.drop(columns=['nulos_totales'], inplace=True)

# Verificar el nuevo tamaño
print("Tamaño después de limpieza:", tabla_final_limpia.shape)


In [ ]:
#Vamos a terminar de preparar nuestra tabla

# 1. crear una columna con el número de hijos totales
tabla_final_limpia['total_kids'] = tabla_final_limpia['Kidhome'] + tabla_final_limpia['Teenhome']

In [ ]:

# 2. crear una buena categorización para el nivel de estudios
tabla_final_limpia['education'].unique()

# Para adaptar el dato a nuestro sistema, vamos a realizar algunas suposiciones. todo basic. equivale a "Educación Primaria" y "professional.course" es "Formación profesional" y 
# el resto traducciones siguiendo el sentido común.
def simplificar_educacion(valor):
    if isinstance(valor, str):
        if 'basic.' in valor:
            return 'Primaria'
        elif valor == 'professional.course':
            return 'Formación profesional'
        elif valor == 'high.school':
            return 'Instituto'
        elif valor == 'university.degree':
            return 'Grado Universitario'
        else:
            return np.nan  # El resto se considera nulo
    return np.nan

# Aplicamos la función a una nueva columna (o la misma si quieres sobrescribir)
tabla_final_limpia['education_simplificada'] = tabla_final_limpia['education'].apply(simplificar_educacion)
tabla_final_limpia.drop(columns=['education'], inplace=True)
tabla_final_limpia.head()


In [ ]:
# 3. crear una buena categorización para el estado civil
tabla_final_limpia['marital'].unique()

def simplificar_marital(valor):
    if isinstance(valor, str):
        if 'MARRIED' in valor:
            return 'Casado'
        elif valor == 'SINGLE':
            return 'Soltero'
        elif valor == 'DIVORCED':
            return 'Divorciado'
        else:
            return np.nan  # El resto se considera nulo
    return np.nan

# Aplicamos la función a una nueva columna (o la misma si quieres sobrescribir)
tabla_final_limpia['estado_civil'] = tabla_final_limpia['marital'].apply(simplificar_marital)
tabla_final_limpia.drop(columns=['marital'], inplace=True)
tabla_final_limpia.head()

In [ ]:
# 4. Desechar columnas que no vaya a usar: he intenado instalar extensiones y ficheros que transformaran coordenadas en ciudades, pero no me ha dado resultado.
# Para una mejor gestión de recursos, también eliminaré 'nr.employed', 'ID', "euribor3m" y "cons.conf.idx" porque no me parece que vayan a dar más información que la fácilmente deducible

tabla_final_limpia.drop(columns=['latitude', 'longitude', 'euribor3m', 'cons.conf.idx', 'nr.employed', 'ID'], inplace=True)
tabla_final_limpia.head(3)

In [ ]:
# Ahora vamos a seguir preparando los datos de mejor forma

traducciones = {
    'Income': 'Ingresos',
    'age': 'Edad',
    'default': 'Incumplimiento pagos',
    'contact': 'Contacto con cliente',
    'duration': 'Duración última llamada',
    'campaign': 'Contactos con cliente',
    'pdays': 'Días desde último contacto',
    'previous': 'Contactos pre-campaña',
    'poutcome':'Resultado campaña anterior',
    'emp.var.rate': 'Tasa variación empleo',
    'cons.price.idx': 'IPC',
    'date': 'Fecha interacción',
    'Dt_Customer': 'Fecha inicio cliente',
    'Kidhome': 'Hijos pequeños',
    'Teenhome': 'Hijos adolescentes',
    'total_kids': 'Total hijos',
    'job': 'Trabajo',
    'marital': 'Estado civil',
    'housing': 'Tiene hipoteca',
    'loan': 'Tiene préstamo',
    'y': 'Respuesta campaña',
    'education_simplificada': 'Nivel educativo',
    'NumWebVisitsMonth': 'Visitas web mes'
}

tabla_final_limpia.rename(columns=traducciones, inplace=True)

tabla_final_limpia.head()

In [ ]:
tabla_final_limpia.to_excel("tabla_final_definitiva.xlsx", index=False)
